In [112]:
with open("/Users/marcus/Documents/GitHub/more-attention/app/scripts/gen_scripts/2text.txt", "r", encoding="utf-8") as f:
  text = f.read()

In [113]:
from thumbnail import generate_thumbnail

thumbnail_script = text[:1000]
thumbnail_path = generate_thumbnail(template_choice=0, script_text=thumbnail_script, font_size=65, line_spacing_px=7, font_weight="bold", thickness_px=0.1, use_ellipsis=True)

Saved thumbnail: /Users/marcus/Downloads/video_thumbnails_reddit1/WRH_white_20250904_130849.png
Words used: 35/191  (ellipsis added: True)


In [92]:
TITLE = "[FULL STORY] You Were Always Her Favorite. Not For Long.” Weeks Later, The Truth Tore Our Family Apart…"
DESCRIPTION = (
  "Daily stories to help with life's emotional damage 🗣🔥🔥\n"

  "We write original first-person dramas inspired by real life—the kinds of twists people actually face. We adapt, condense, and sometimes combine elements to build tight, high-stakes stories with fresh characters and sharp pacing. Names, details, and timelines are changed. These are our own scripts, crafted to entertain. Hit play and enjoy.\n"

  "Keywords :)\n"
  "reddit relationship stories, boyfriend cheating reddit, infidelity on reddit, open relationship regrets reddit, revenge relationship stories, aita reddit tales relationships, messiest reddit relationships, wild reddit dating stories, storytime aita reddit, relationship advice from reddit, reddit story compilation, tiktok reddit compilations, cheating girlfriend reddit, surviving infidelity reddit, cheating boyfriend stories, ex girlfriend reddit drama, reddit boyfriend saga, reconciliation stories reddit, youtube reddit infidelity, cheating in relationships story, reddit girlfriend stories, open marriage gone wrong, entitled parents stole my birthday, entitled sister saga, wedding day betrayal story, cheating in a relationship tales, infidelity and betrayal reddit, cheating complications reddit, relationships cheating revenge reddit, cheating stories with updates, cheating husband reddit stories, cheating spouse on reddit, best reddit love tales, reddit out loud relationships, family feud on reddit, parenting struggles and fixes, sibling rivalry solutions, family secrets revealed, coping with family conflict, multigenerational family dynamics, toxic family members advice, family therapy moments, betrayal inside the family, forgiveness after betrayal, rebuild trust in family, betrayal quotes and lines, betrayal trauma recovery tips, betrayal in books discussion, share your betrayal experience, surviving betrayal and moving on, dealing with betrayal emotions, signs of betrayal in love, friendship betrayal storytime, petty revenge is sweet, revenge served cold, revenge on ex partner, ultimate revenge stories, reddit relationship advice, family drama reddit, true reddit relationship tales, top reddit stories, best of reddit stories, reddit relationship horror, reddit family disputes, reddit family secrets, reddit confessions family, heartwarming reddit tales, reddit love stories, requestedreads, ami the jerk aita, amithejerk updates, reddit update part 2, relationship update arc, r/relationship_advice drama, best reddit narrations, theredditnarrator inspired, telltales channel vibes, secretvoices style, inkspiller stories, qley narration, top reddit reads of all time, crazy relationship stories reddit, open relationship fallout, polyamory regret reddit, caught cheating reddit, karma revenge reddit, nuclear revenge relationship, petty revenge relationship, relationship red flags reddit, going no contact family, gaslighting recovery reddit, messy breakup stories, ex exposed reddit, caught in 4k cheating, cheating spouse revenge plan, divorce court reddit story, custody drama reddit, entitled parents meltdown, entitled siblings reddit, golden child vs scapegoat, will reading gone wrong, inheritance drama reddit, workplace affair reddit, coworker cheating saga, roommate from hell reddit, neighbor drama reddit, best aita relationship, aita marriage drama, aita family conflict, aita updates compilation, reddit tea time, reddit storytelling channel, reddit narration long form, reddit shorts relationship, youtube shorts aita, reddit tts stories, viral reddit drama, trending reddit relationships, real life inspired stories, first person reddit-style, dramatic storytime compilation"

)
HASHTAGS = "#redditfamilydrama #reddit #redditrelationship"  # Hashtags go in title/description text
TAGS = ["AITA", "Reddit", "storytime", "#redditstories",
        "#redditconfessions", "#redditreads", "#redditandchill", "#storytellingenglishforsleeping", "#relationshipstories", "#redditfamilydrama", "#entitledparents",
        "#aita", "#aitareddit"
    ]
SCHEDULE_AT_LOCAL = None #"2025-09-04 19:30"
# Choose a mode: "instant" | "scheduled" | "private"
MODE = "private"

In [94]:
from upload_yt import upload_youtube

In [ ]:
combined_yes_captions_path = "path/to/combined_yes_captions"
thumbnail_path = "path/to/thumbnail"

In [ ]:
post_youtube = input("Do you want to post this to Youtube using API?")
if post_youtube.lower() == "y":
    upload_youtube(combined_yes_captions_path, thumbnail_path, TITLE, DESCRIPTION, HASHTAGS, TAGS, MODE, SCHEDULE_AT_LOCAL)
else:
    print("Not uploading to YouTube.")
    pass

In [16]:
from editing import scroll_left_incremental
import time
start_s = 6
time.sleep(2.0)

scroll_left_incremental(start=(500, 700), pixels=115, steps=start_s, pause=0.0005)

In [1]:
from upload_yt2 import upload_youtube2

VIDEO_PATH = "/Users/marcus/Downloads/reddit1_filmora_captioned/exported_2025-09-05_19-48-41_My_Video.mp4"
THUMB_PATH = "/Users/marcus/Downloads/video_thumbnails_reddit1/WRH_white_20250905_200323.png"

TITLE = "[FULL STORY] My Dad Said He “Couldn’t Afford” College for Me, But I Saw Where the Money Went"

DESCRIPTION = "\n".join([
    "Stories with more chaos than Mario Kart on Rainbow Road 🎮🌈💥",
    "",
    "",
    "We write original first person dramas inspired by real life "
    "We adapt, condense, and sometimes combine elements to build tight, high-stakes stories with fresh "
    "characters and sharp pacing. Names, details, and timelines are changed. These are our own scripts, "
    "crafted to entertain. Hit play and enjoy."
])

HASHTAGS = "#aita  #reddit #redditstoriesfullstory   #redditconfessions  #relationshipstories   #redditfamilydrama #redditreadings #betrayal #relationshipdrama"
TAGS = ["redditfamilydrama", "reddit", "redditrelationship"]
SCHEDULE_AT_LOCAL = None #"2025-09-04 19:30"
# Choose a mode: "instant" | "scheduled" | "private"
MODE = "private"

upload_youtube2(VIDEO_PATH, THUMB_PATH, TITLE, DESCRIPTION, HASHTAGS, TAGS, MODE, SCHEDULE_AT_LOCAL)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=491585816596-a6h72et0b0c13phbet1j2q14hvbv3s1n.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A51189%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.upload+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube&state=YQI3pJK6COjhuCojafXyx8fbGkQWCf&access_type=offline
Starting upload: exported_2025-09-05_19-48-41_My_Video.mp4  (2818.4 MB)
  → Upload progress: 0%
  → Upload progress: 0%
  → Upload progress: 0%
  → Upload progress: 1%
  → Upload progress: 1%
  → Upload progress: 1%
  → Upload progress: 2%
  → Upload progress: 2%
  → Upload progress: 2%
  → Upload progress: 2%
  → Upload progress: 3%
  → Upload progress: 3%
  → Upload progress: 3%
  → Upload progress: 4%
  → Upload progress: 4%
  → Upload progress: 4%
  → Upload progress: 5%
  → Upload progress: 5%
  → Upload progress: 5%
  → Upload progress: 5%
  → Upload progress: 6%
  → Upload

'XiEiJN8FMH0'

In [1]:
print("hi")

hi


In [2]:
from kokoro import KPipeline